In [1]:
import re
import spacy
import pandas as pd 
from unidecode import unidecode
from collections import Counter
nlp = spacy.blank("es")

In [2]:
def Limpiar_texto(texto):
    texto = unidecode(texto)
    # Eliminar cualquier signo de puntuación
    puntuación = r'[.,;:¡!¿?@#$%&[\](){}<>~=+\*/|\\_^`"\']'
    #puntuación = r'[^\w\s]|(?<=\w)\.(?=[a-zA-Z])'
    texto = re.sub(puntuación, '', texto)

    # Eliminar cualquier carácter que no sea una letra o un número
    alfanumérico = r'\w+(?:-\w+)*'
    texto = re.sub(alfanumérico, lambda x: x.group(0).replace('-', ' '), texto)

    # Eliminar cualquier caracter que no sea una letra o un número o un espacio en blanco
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    # eliminar espacios en blanco iniciales
    texto = re.sub(r"^\s+", "", texto)
    doc = nlp(texto.lower())

    return " ".join(unidecode(token.norm_) for token in doc if not token.is_punct)


In [3]:
accionados = pd.read_excel('accionado.xlsx')
accionados['texto_limpio'] = accionados['Demandado'].apply(Limpiar_texto)    
accionados.head(20)

,id,Demandado,tipo demandado,OK,Demandado_1,Demandado_2,texto_limpio
0,1,"ALCALDA DE SANTA MARTA, SECRETARIA DE EDUCACIO...",ALCALDIA,OK,ALCALDIA,SECRETARIA DE EDUCACIÓN,alcalda de santa marta secretaria de educacion...
1,2,JUZGADO 5 LABORAL DE CIRCUITO DE CARTAGENA,JUZGADO,NaN,NaN,NaN,juzgado 5 laboral de circuito de cartagena
2,3,JUZGADO 5 LABORAL DE CIRCUITO DE CARTAGENA,JUZGADO,NaN,NaN,NaN,juzgado 5 laboral de circuito de cartagena
3,4,ETB DE BOGOTA Y OTROS,OTROS*-*,NaN,NaN,NaN,etb de bogota y otros
4,5,ADMINISTRADORA COLOMBIANA DE PENSIONES COLPENS...,AFP,OK,COLPENSIONES,NaN,administradora colombiana de pensiones colpens...
5,6,ASMET SALUD EPS,EPS,OK,ASMET SALUD EPS,NaN,asmet salud eps
6,7,ASMET SALUD EPS,EPS,OK,ASMET SALUD EPS,NaN,asmet salud eps
7,8,LIBRERIA ONLINE COLOMBIA,OTROS*-*,NaN,NaN,NaN,libreria online colombia
8,9,MUNDIAL DE SEGUROS S.A.,SEGUROS,NaN,NaN,NaN,mundial de seguros sa
9,10,ALCALDIA DEL MUNICIPIO DE SAN MARTIN CESAR Y O...,ALCALDIA,OK,ALCALDIA MUNICIPAL,NaN,alcaldia del municipio de san martin cesar y otro


# Etiqueta_Otros

In [4]:
otros = r'y\s?otro(s)?'
accionados = accionados.assign(otros=accionados['Demandado'].astype(str).str.contains(otros, flags=re.IGNORECASE).astype(int))
accionados['texto_limpio'] = accionados['texto_limpio'].str.replace(otros, ' ', regex=True)

C:\Users\Equipo\AppData\Local\Temp\ipykernel_13360\3864031245.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  accionados = accionados.assign(otros=accionados['Demandado'].astype(str).str.contains(otros, flags=re.IGNORECASE).astype(int))


# Stop_words

In [5]:
stop_words = ["sa", "sas", "ltda", "y", "e", "ni", "que", "pero", "si", "porque", "cuando", "pues", "sino",
            "aunque","el","los","las","lo","la", "un","una","unos","unas","al","del",
            "a","ante","bajo","cabe","con","contra","de","desde",
            "en","entre","hacia","hasta","para",
            "por" ,"segun" ,"sin" ,"so" ,"sobre" ,"tras"]
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(r'(\b\w\s){2,}\w\b', lambda x: x.group().replace(' ', ''), x))
#letras_solas = r'\b[a-zA-Z]\b'
#accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(letras_solas, '', x))
patron = r"\b(" + "|".join(stop_words) + r")\b"
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(patron,"",x))
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(' +', ' ', x))
# eliminar espacios en blanco iniciales
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(r"^\s+", "",x))

# Estandarizado

In [6]:
import re

#departamentos_municipios = ['abejorral', 'abrego', 'abriaqui', 'acacias', 'acandi', 'acevedo', 'achi', 'agrado', 'agua de dios', 'aguachica', 'aguada', 'aguadas', 'aguazul', 'agustin codazzi', 'aipe', 'alban', 'albania', 'alcala', 'aldana', 'alejandria', 'algarrobo', 'algeciras', 'almaguer', 'almeida', 'alpujarra', 'altamira', 'alto baudo', 'altos del rosario', 'alvarado', 'amaga', 'amalfi', 'amazonas', 'ambalema', 'anapoima', 'ancuya', 'andalucia', 'andes', 'angelopolis', 'angostura', 'anolaima', 'anori', 'anserma', 'ansermanuevo', 'antioquia', 'anza', 'anzoategui', 'apartado', 'apia', 'apulo', 'aquitania', 'aracataca', 'aranzazu', 'aratoca', 'arauca', 'arauquita', 'arbelaez', 'arboleda', 'arboledas', 'arboletes', 'arcabuco', 'arenal', 'argelia', 'ariguani', 'arjona', 'armenia', 'armero', 'arroyohondo', 'astrea', 'ataco', 'atlantico', 'atrato', 'ayapel', 'bagado', 'bahia solano', 'bajo baudo', 'balboa', 'baranoa', 'baraya', 'barbacoas', 'barbosa', 'barichara', 'barranca de upia', 'barrancabermeja', 'barrancas', 'barranco de loba', 'barranco minas', 'barranquilla', 'becerril', 'belalcazar', 'belen', 'belen de bajira', 'belen de los andaquies', 'belen de umbria', 'bello', 'belmira', 'beltran', 'berbeo', 'betania', 'beteitiva', 'betulia', 'bituima', 'boavita', 'bochalema', 'bogota', 'bojaca', 'bojaya', 'bolivar', 'bosconia', 'boyaca', 'briceno', 'bucaramanga', 'bucarasica', 'buena vista', 'buenaventura', 'buenavista', 'buenos aires', 'buesaco', 'bugalagrande', 'buritica', 'busbanza', 'cabrera', 'cabuyaro', 'cacahual', 'caceres', 'cachipay', 'cachira', 'cacota', 'caicedo', 'caicedonia', 'caimito', 'cajamarca', 'cajibio', 'cajica', 'calamar', 'calarca', 'caldas', 'caldono', 'cali', 'california', 'calima', 'caloto', 'campamento', 'campo de la cruz', 'campoalegre', 'campohermoso', 'canalete', 'canasgordas', 'candelaria', 'cantagallo', 'caparrapi', 'capitanejo', 'caqueta', 'caqueza', 'caracoli', 'caramanta', 'carcasi', 'carepa', 'carmen de apicala', 'carmen de carupa', 'carmen del darien', 'carolina', 'cartagena', 'cartagena del chaira', 'cartago', 'caruru', 'casabianca', 'casanare', 'castilla la nueva', 'cauca', 'caucasia', 'cepita', 'cerete', 'cerinza', 'cerrito', 'cerro san antonio', 'certegui', 'cesar', 'chachagui', 'chaguani', 'chalan', 'chameza', 'chaparral', 'charala', 'charta', 'chia', 'chigorodo', 'chima', 'chimichagua', 'chinacota', 'chinavita', 'chinchina', 'chinu', 'chipaque', 'chipata', 'chiquinquira', 'chiquiza', 'chiriguana', 'chiscas', 'chita', 'chitaga', 'chitaraque', 'chivata', 'chivolo', 'chivor', 'choachi', 'choco', 'choconta', 'cicuco', 'cienaga', 'cienaga de oro', 'cienega', 'cimitarra', 'circasia', 'cisneros', 'ciudad bolivar', 'clemencia', 'cocorna', 'coello', 'cogua', 'colon', 'coloso', 'combita', 'concepcion', 'concordia', 'condoto', 'confines', 'consaca', 'contadero', 'contratacion', 'convencion', 'copacabana', 'coper', 'cordoba', 'corinto', 'coromoro', 'corozal', 'corrales', 'cota', 'cotorra', 'covarachia', 'covenas', 'coyaima', 'cravo norte', 'cuaspud', 'cubara', 'cubarral', 'cucaita', 'cucunuba', 'cucuta', 'cucutilla', 'cuitiva', 'cumaral', 'cumaribo', 'cumbal', 'cumbitara', 'cunday', 'cundinamarca', 'curillo', 'curiti', 'curumani', 'dabeiba', 'dagua', 'dibula', 'distraccion', 'dolores', 'don matias', 'dosquebradas', 'duitama', 'durania', 'ebejico', 'el aguila', 'el bagre', 'el banco', 'el cairo', 'el calvario', 'el canton del san pablo', 'el carmen', 'el carmen de atrato', 'el carmen de bolivar', 'el carmen de chucuri', 'el carmen de viboral', 'el castillo', 'el cerrito', 'el charco', 'el cocuy', 'el colegio', 'el copey', 'el doncello', 'el dorado', 'el dovio', 'el encanto', 'el espino', 'el guacamayo', 'el guamo', 'el litoral del san juan', 'el molino', 'el paso', 'el paujil', 'el penol', 'el penon', 'el pinon', 'el playon', 'el reten', 'el retorno', 'el roble', 'el rosal', 'el rosario', 'el santuario', 'el tablon de gomez', 'el tambo', 'el tarra', 'el zulia', 'elias', 'encino', 'enciso', 'entrerrios', 'envigado', 'espinal', 'facatativa', 'falan', 'filadelfia', 'filandia', 'firavitoba', 'flandes', 'florencia', 'floresta', 'florian', 'florida', 'floridablanca', 'fomeque', 'fonseca', 'fortul', 'fosca', 'francisco pizarro', 'fredonia', 'fresno', 'frontino', 'fuente de oro', 'fundacion', 'funes', 'funza', 'fuquene', 'fusagasuga', 'gachala', 'gachancipa', 'gachantiva', 'gacheta', 'galan', 'galapa', 'galeras', 'gama', 'gamarra', 'gambita', 'gameza', 'garagoa', 'garzon', 'genova', 'gigante', 'ginebra', 'giraldo', 'girardot', 'girardota', 'giron', 'gomez plata', 'gonzalez', 'gramalote', 'granada', 'guaca', 'guacamayas', 'guacari', 'guachene', 'guacheta', 'guachucal', 'guadalajara de buga', 'guadalupe', 'guaduas', 'guainia', 'guaitarilla', 'gualmatan', 'guamal', 'guamo', 'guapi', 'guapota', 'guaranda', 'guarne', 'guasca', 'guatape', 'guataqui', 'guatavita', 'guateque', 'guatica', 'guavata', 'guaviare', 'guayabal de siquima', 'guayabetal', 'guayata', 'guepsa', 'guican', 'gutierrez', 'hacari', 'hatillo de loba', 'hato', 'hato corozal', 'hatonuevo', 'heliconia', 'herran', 'herveo', 'hispania', 'hobo', 'honda', 'huila', 'ibague', 'icononzo', 'iles', 'imues', 'inirida', 'inza', 'ipiales', 'iquira', 'isnos', 'istmina', 'itagui', 'ituango', 'iza', 'jambalo', 'jamundi', 'jardin', 'jenesano', 'jerico', 'jerusalen', 'jesus maria', 'jordan', 'juan de acosta', 'junin', 'jurado', 'la apartada', 'la argentina', 'la belleza', 'la calera', 'la capilla', 'la ceja', 'la celia', 'la chorrera', 'la cruz', 'la cumbre', 'la dorada', 'la esperanza', 'la estrella', 'la florida', 'la gloria', 'la guadalupe', 'la guajira', 'la jagua de ibirico', 'la jagua del pilar', 'la llanada', 'la macarena', 'la merced', 'la mesa', 'la montanita', 'la palma', 'la paz', 'la pedrera', 'la pena', 'la pintada', 'la plata', 'la playa', 'la primavera', 'la salina', 'la sierra', 'la tebaida', 'la tola', 'la union', 'la uvita', 'la vega', 'la victoria', 'la virginia', 'labateca', 'labranzagrande', 'landazuri', 'lebrija', 'leguizamo', 'leiva', 'lejanias', 'lenguazaque', 'lerida', 'leticia', 'libano', 'liborina', 'linares', 'lloro', 'lopez', 'lorica', 'los andes', 'los cordobas', 'los palmitos', 'los patios', 'los santos', 'lourdes', 'luruaco', 'macanal', 'macaravita', 'maceo', 'macheta', 'madrid', 'magangue', 'magdalena', 'magui', 'mahates', 'maicao', 'majagual', 'malaga', 'malambo', 'mallama', 'manati', 'manaure', 'mani', 'manizales', 'manta', 'manzanares', 'mapiripan', 'mapiripana', 'margarita', 'maria la baja', 'marinilla', 'maripi', 'mariquita', 'marmato', 'marquetalia', 'marsella', 'marulanda', 'matanza', 'medellin', 'medina', 'medio atrato', 'medio baudo', 'medio san juan', 'melgar', 'mercaderes', 'mesetas', 'meta', 'milan', 'miraflores', 'miranda', 'miriti parana', 'mistrato', 'mitu', 'mocoa', 'mogotes', 'molagavita', 'momil', 'mompos', 'mongua', 'mongui', 'moniquira', 'monitos', 'montebello', 'montecristo', 'montelibano', 'montenegro', 'monteria', 'monterrey', 'morales', 'morelia', 'morichal', 'morroa', 'mosquera', 'motavita', 'murillo', 'murindo', 'mutata', 'mutiscua', 'muzo', 'narino', 'nataga', 'natagaima', 'nechi', 'necocli', 'neira', 'neiva', 'nemocon', 'nilo', 'nimaima', 'nobsa', 'nocaima', 'norcasia', 'norosi', 'norte santander', 'novita', 'nueva granada', 'nuevo colon', 'nunchia', 'nuqui', 'obando', 'ocamonte', 'ocana', 'oiba', 'oicata', 'olaya', 'olaya herrera', 'onzaga', 'oporapa', 'orito', 'orocue', 'ortega', 'ospina', 'otanche', 'ovejas', 'pachavita', 'pacho', 'pacoa', 'pacora', 'padilla', 'paez', 'paicol', 'pailitas', 'paime', 'paipa', 'pajarito', 'palermo', 'palestina', 'palmar', 'palmar de varela', 'palmas del socorro', 'palmira', 'palmito', 'palocabildo', 'pamplona', 'pamplonita', 'pana pana', 'pandi', 'panqueba', 'papunahua', 'paramo', 'paratebueno', 'pasca', 'pasto', 'patia', 'pauna', 'paya', 'paz de ariporo', 'paz de rio', 'pedraza', 'pelaya', 'penol', 'pensilvania', 'peque', 'pereira', 'pesca', 'piamonte', 'piedecuesta', 'piedras', 'piendamo', 'pijao', 'pijino del carmen', 'pinchote', 'pinillos', 'piojo', 'pisba', 'pital', 'pitalito', 'pivijay', 'planadas', 'planeta rica', 'plato', 'policarpa', 'polonuevo', 'ponedera', 'popayan', 'pore', 'potosi', 'pradera', 'prado', 'providencia', 'pueblo bello', 'pueblo nuevo', 'pueblo rico', 'pueblo viejo', 'pueblorrico', 'puente nacional', 'puerres', 'puerto alegria', 'puerto arica', 'puerto asis', 'puerto berrio', 'puerto boyaca', 'puerto caicedo', 'puerto carreno', 'puerto colombia', 'puerto concordia', 'puerto escondido', 'puerto gaitan', 'puerto guzman', 'puerto libertador', 'puerto lleras', 'puerto lopez', 'puerto nare', 'puerto narino', 'puerto parra', 'puerto rico', 'puerto rondon', 'puerto salgar', 'puerto santander', 'puerto tejada', 'puerto triunfo', 'puerto wilches', 'puli', 'pupiales', 'purace', 'purificacion', 'purisima', 'putumayo', 'quebradanegra', 'quetame', 'quibdo', 'quimbaya', 'quinchia', 'quindio', 'quipama', 'quipile', 'ragonvalia', 'ramiriqui', 'raquira', 'recetor', 'regidor', 'remedios', 'remolino', 'repelon', 'restrepo', 'retiro', 'ricaurte', 'rio blanco', 'rio de oro', 'rio iro', 'rio quito', 'rio viejo', 'riofrio', 'riohacha', 'rionegro', 'riosucio', 'risaralda', 'rivera', 'roberto payan', 'roldanillo', 'roncesvalles', 'rondon', 'rosas', 'rovira', 'sabana de torres', 'sabanagrande', 'sabanalarga', 'sabanas de san angel', 'sabaneta', 'saboya', 'sacama', 'sachica', 'sahagun', 'saladoblanco', 'salamina', 'salazar', 'saldana', 'salento', 'salgar', 'samaca', 'samana', 'samaniego', 'sampues', 'san agustin', 'san alberto', 'san andres', 'san andres de cuerquia', 'san andres de tumaco', 'san andres sotavento', 'san andres y providencia', 'san antero', 'san antonio', 'san antonio del tequendama', 'san benito', 'san benito abad', 'san bernardo', 'san bernardo del viento', 'san calixto', 'san carlos', 'san carlos de guaroa', 'san cayetano', 'san cristobal', 'san diego', 'san eduardo', 'san estanislao', 'san felipe', 'san fernando', 'san francisco', 'san gil', 'san jacinto', 'san jacinto del cauca', 'san jeronimo', 'san joaquin', 'san jose', 'san jose de la montana', 'san jose de miranda', 'san jose de pare', 'san jose de ure', 'san jose del fragua', 'san jose del guaviare', 'san jose del palmar', 'san juan de arama', 'san juan de betulia', 'san juan de rio seco', 'san juan de uraba', 'san juan del cesar', 'san juan nepomuceno', 'san juanito', 'san lorenzo', 'san luis', 'san luis de gaceno', 'san luis de since', 'san marcos', 'san martin', 'san martin de loba', 'san mateo', 'san miguel', 'san miguel de sema', 'san onofre', 'san pablo', 'san pablo de borbur', 'san pedro', 'san pedro de cartago', 'san pedro de uraba', 'san pelayo', 'san rafael', 'san roque', 'san sebastian', 'san sebastian de buenavista', 'san vicente', 'san vicente de chucuri', 'san vicente del caguan', 'san zenon', 'sandona', 'santa ana', 'santa barbara', 'santa barbara de pinto', 'santa catalina', 'santa helena del opon', 'santa isabel', 'santa lucia', 'santa maria', 'santa marta', 'santa rosa', 'santa rosa de cabal', 'santa rosa de osos', 'santa rosa de viterbo', 'santa rosa del sur', 'santa rosalia', 'santa sofia', 'santacruz', 'santafe de antioquia', 'santana', 'santander', 'santander de quilichao', 'santiago', 'santiago de tolu', 'santo domingo', 'santo tomas', 'santuario', 'sapuyes', 'saravena', 'sardinata', 'sasaima', 'sativanorte', 'sativasur', 'segovia', 'sesquile', 'sevilla', 'siachoque', 'sibate', 'sibundoy', 'silos', 'silvania', 'silvia', 'simacota', 'simijaca', 'simiti', 'sincelejo', 'sipi', 'sitionuevo', 'soacha', 'soata', 'socha', 'socorro', 'socota', 'sogamoso', 'solano', 'soledad', 'solita', 'somondoco', 'sonson', 'sopetran', 'soplaviento', 'sopo', 'sora', 'soraca', 'sotaquira', 'sotara', 'suaita', 'suan', 'suarez', 'suaza', 'subachoque', 'sucre', 'suesca', 'supata', 'supia', 'surata', 'susa', 'susacon', 'sutamarchan', 'sutatausa', 'sutatenza', 'tabio', 'tado', 'talaigua nuevo', 'tamalameque', 'tamara', 'tame', 'tamesis', 'taminango', 'tangua', 'taraira', 'tarapaca', 'taraza', 'tarqui', 'tarso', 'tasco', 'tauramena', 'tausa', 'tello', 'tena', 'tenerife', 'tenjo', 'tenza', 'teorama', 'teruel', 'tesalia', 'tibacuy', 'tibana', 'tibasosa', 'tibirita', 'tibu', 'tierralta', 'timana', 'timbio', 'timbiqui', 'tinjaca', 'tipacoque', 'tiquisio', 'titiribi', 'toca', 'tocaima', 'tocancipa', 'togui', 'toledo', 'tolima', 'tolu viejo', 'tona', 'topaga', 'topaipi', 'toribio', 'toro', 'tota', 'totoro', 'trinidad', 'trujillo', 'tubara', 'tuchin', 'tulua', 'tunja', 'tunungua', 'tuquerres', 'turbaco', 'turbana', 'turbo', 'turmeque', 'tuta', 'tutaza', 'ubala', 'ubaque', 'ulloa', 'umbita', 'une', 'unguia', 'union panamericana', 'uramita', 'uribe', 'uribia', 'urrao', 'urumita', 'usiacuri', 'utica', 'valdivia', 'valencia', 'valle cauca', 'valle de guamez', 'valle de san jose', 'valle de san juan', 'valledupar', 'valparaiso', 'vaupes', 'vegachi', 'velez', 'venadillo', 'venecia', 'ventaquemada', 'vergara', 'versalles', 'vetas', 'viani', 'vichada', 'victoria', 'vigia del fuerte', 'vijes', 'villa caro', 'villa de leyva', 'villa de san diego de ubate', 'villa del rosario', 'villa rica', 'villagarzon', 'villagomez', 'villahermosa', 'villamaria', 'villanueva', 'villapinzon', 'villarrica', 'villavicencio', 'villavieja', 'villeta', 'viota', 'viracacha', 'vista hermosa', 'viterbo', 'yacopi', 'yacuanquer', 'yaguara', 'yali', 'yarumal', 'yavarate', 'yolombo', 'yondo', 'yopal', 'yotoco', 'yumbo', 'zambrano', 'zapatoca', 'zapayan', 'zaragoza', 'zarzal', 'zetaquira', 'zipacon', 'zipaquira', 'zona bananera']
departamentos = ["amazonas", "antioquia", "arauca", "atlantico", "bolivar", "boyaca", "caldas", "caqueta", "casanare", "cauca", "cesar", "choco", "cordoba", "cundinamarca", "guainia", "guaviare", "huila", "guajira", "magdalena", "meta", "narino", "norte santander", "putumayo", "quindio", "risaralda", "san andres providencia", "santander", "sucre", "tolima", "valle", "valle cauca", "vaupes", "vichada"]

#Esta biblioteca contiene las regex de 
entidades = {
        "uariv": r"^(?!.*\bunp\b)(?=.*unidad)(?=.*victimas).*|(uariv).*$",
        #alcaldias
        "alcaldia": r"alcaldia(s)?",
        "gobernacion": r"gobernacion",
        #Pensiones
        "colpensiones": r"colpension(es)?|^(?=.*administradora)(?=.*colombiana)(?=.*pensiones).*",
        "colfondos afp": r"colfondo(s)?",
        "skandia afp": r"skandia",
        "proteccion afp": r"^(?!.*(ministerio|contribucion(es)?|alfa|nacional|social|policia|unp|inspeccion|eps|unidad|seguridad|grupo|secretaria|empleo|empleados)).*proteccion|(?=.*proteccion)(?=.*afp).*$",
        "porvenir afp": r"^(?!.*(alcaldia|clinica|condominio|sendero(s)?|secretaria|inspeccion|comunal|conjunto|junta|centro|comercial|asociacion|consejo|inspeccion|residencial|condado(s)?)).*porvenir|(?=.*porvenir)(?=.*afp).*$",
        
        #Eps
        "asmet salud eps": r"asme(.*)[dt]",
        "asociacion indigena cauca aic eps": r"\baic\b|^(?=.*asociacion)(?=.*indigena)(?=.*cauca).*",
        "aliansalud eps": r"(alian\s?salud|alianza salud)",
        "ambuq eps": r"ambuq|^(?=.*asociacion)(?=.*mutual)(?=.*barrios)(?=.*unidos)(?=.*quibdo).*",
        "anas wayuu eps": r"^(?=.*anas)(?=.*wayu(u)?).*",
        "cajacopi eps": r"^(?=.*caja\s?copi)(?=.*eps)",
        "capital salud eps": r"(?=.*capital)(?=.*salud)|(?=.*capital)(?=.*eps)",
        "capresoca eps": r"capresoca",
        "coomeva eps": r"(?=.*co(o)?meva)(?=.*eps)",
        "coosalud eps": r"co(o)?salud",
        "comparta eps": r"(?=.*comparta)(?=.*eps)",
        "compensar eps": r"(?=.*compensar)(?=.*eps)",
        "comfachoco eps": r"(?=.*comfa\s?choco)(?=.*eps)",
        "comfaca eps": r"(?=.*comfa\s?ca)(?=.*eps)",
        "comfacor eps": r"(?=.*comfa\s?cor)(?=.*eps)",
        "comfasucre eps": r"(?=.*comfa\s?sucre)(?=.*eps)",
        "comfanorte eps": r"(?=.*comfa\s?norte)(?=.*eps)",
        "comfaoriente eps": r"(?=.*comfa\s?oriente)(?=.*eps)",
        "comfamiliar eps": r"(?=.*comfamiliar)(?=.*eps)",
        "comfenalco eps": r"(?=.*comfenalco)(?=.*eps)",
        "comfandi eps": r"(?=.*comfandi)(?=.*eps)",
        "convida eps": r"convida", 
        "cosmitet eps": r"cosmitet",
        "dusakawi eps": r"dusakawi",
        "ecoopsos eps": r"ecoopsos",
        "emssanar eps": r"ems(s)?anar",
        "emcosalud eps": r"(?=.*emco\s?salud)(?=.*eps)",
        "famisanar eps": r"famisanar",
        "familiar colombia eps": r"(?=.*familiar)(?=.*colombia)(?=.*eps)",
        "mallamas eps": r"(?=.*mal(l)?a\s?ma(s)?)(?=.*eps)",
        "medimas eps": r"medimas",
        "mutualser eps": r"mutual\s?ser",
        "nueva eps": r"(?=.*nueva)(?=.*eps)|(?=.*empresa)(?=.*salud)(?=.*nueva)",
        "pijaos salud eps": r"(?=.*pijao(s)?)(?=.*salud)",
        "saludcoop": r"saludco(o)?p",
        "salud bolivar eps": r"(?=.*salud\s?bolivar)(?=.*eps)",
        "salud mia eps": r"salud\s?mia",
        "salud total eps": r"salud\s?total",
        "sanitas eps": r"\bsanita(s)?\b|(?=.*colsanita(s)?)(?=.*eps)",
        "savia salud eps": r"(?=.*alianza)(?=.*medellin)(?=.*antioquia)|savia",
        "servicio occidental de salud eps": r"(?=.*\bsos\b)(?=.*eps(i)?)|(?=.*servicio)(?=.*occidental)(?=.*salud)",
        "sura eps": r"(?=.*sura)(?=.*eps)|(?=.*entidad)(?=.*salud)(?=.*sura)",
        #adres 
    
        #ARL
        "axa colpatria arl": r"(?=.*colpatria)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*colpatria)|(?=.*colpatria)(?=.*axa)",
        "colmena arl": r"(?=.*colmena)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*colmena)",
        "equidad seguros arl": r"(?=.*equidad)(?=.*arl)(?=.*seguro(s)?)|(?=.*administradora)(?=.*laborales)(?=.*equidad)(?=.*seguro(s)?)",
        "seguros alfa arl": r"(?=.*alfa)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*alfa)",
        "arl positiva": r"(?=.*positiva)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*positiva)",
        "arl seguros bolivar": r"(?=.*bolivar)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*bolivar)",
        "sura arl": r"(?=.*sura)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*sura)",
        "arl aurora seguros de vida":  r"(?=.*aurora)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*aurora)",   
    
        #ministerios 
        "ministerio del interior": r"(?=.*ministerio)(?=.*interior)|min\s?interior",
        "ministerio de relaciones exteriores": r"(?=.*ministerio)(?=.*relaciones)|cancilleria",
        "ministerio de hacienda": r"(?=.*ministerio)(?=.*hacienda)|min\s?hacienda", 
        "ministerio de justicia y del derecho": r"(?=.*ministerio)(?=.*justicia)|min\s?justicia",
        "ministerio de defensa": r"(?=.*ministerio)(?=.*defensa)|min\s?defensa",  
        "ministerio de agricultura": r"(?=.*ministerio)(?=.*agricultura)|min\s?agricultura",  
        "ministerio de salud": r"(?=.*ministerio)(?=.*salud)|min\s?salud",
        "ministerio de trabajo": r"(?=.*ministerio)(?=.*trabajo)|min\s?trabajo",
        "ministerio de minas y energia":  r"(?=.*ministerio)(?=.*minas)|min\s?energia",
        "ministerio de industria y comercio": r"(?=.*ministerio)(?=.*comercio)|(?=.*ministerio)(?=.*industria)|(?=.*ministerio)(?=.*turismo)|min\s?cit|min\s?comercio",
        "ministerio de educacion" : r"(?=.*ministerio)(?=.*educacion)|min\s?educacion",
        "ministerio de ambiente": r"(?=.*ministerio)(?=.*ambiente)|min\s?ambiente",
        "ministerio de vivienda": r"(?=.*ministerio)(?=.*vivienda)|min\s?vivienda",
        "ministerio de las tic": r"(?=.*ministerio)(?=.*tecnologia(s)?)|min\s?tic|(?=.*ministerio)(?=.*\btic\b)",
        "ministerio de transporte":  r"(?=.*ministerio)(?=.*transporte)|min\s?transporte",
        "ministerio de cultura":  r"(?=.*ministerio)(?=.*\bcultura\b)|min\s?cultura",
        "ministerio de ciencia tecnologia e innovacion": r"(?=.*ministerio)(?=.*ciencia)|min\s?ciencia",
        "ministerio de deporte": r"(?=.*ministerio)(?=.*deporte)|min\s?deporte",
        "ministerio de igualdad": r"(?=.*ministerio)(?=.*igualdad)|min\s?igualdad|min\s?equidad",
    
        #IAS
        #fiscalia
        #contraloria
        #procuraduria
        "registraduria nacional" : r"registraduria", 
        "migracion colombia": r"(?=.*migra[cs]?ion)(?=.*colombia)|(?=.*unidad)(?=.*migra[cs]?ion)", 
        "dian": r"\bdian\b|(?=.*direccion)(?=.*impuestos)|(?=.*direccion)(?=.*aduanas)",
        "comision nacional servicio civil": r"cnsc|(?=.*comi[sc]?ion)(?=.*servicio)(?=.*civil)|(?=.*servicio)(?=.*civil)(?=.*nacional)",
        #fuerzas_armadas
        #"direccion sanidad ejercito nacional"
        #"direccion sanidad armada nacional"
        #"direccion sanidad policia nacional"
        #"direccion general policia nacional"
        #"direccion general sanidad militar"
        #"ejercito nacional colombia"
    
    
    
    
        #Bancos
        'bancolombia': r"bancolombia", 
        'banco de bogota': r"(?=.*banco)(?=.*bogota)", 
        'banco davivienda': r"davivienda", 
        'banco bbva': r"bbva|(?=.*banco)(?=.*bilbao)(?=.*argentina)", 
        'banco de occidente': r"(?=.*banco)(?=.*occidente)", 
        'banco scotiabank colpatria': r"(?=.*banco)(?=.*colpatria)|scotiabank", 
        'banco itau': r"itau", 
        'banco gnb sudameris': r"(?=.*banco)(?=.*gnb)|(?=.*gnb)(?=.*sudameris)", 
        'banco agrario': r"(?=.*banco)(?=.*agrario)", 
        'banco popular': r"(?=.*banco)(?=.*popular)", 
        'banco caja social': r"banco\s+caja\s+social|bcsc", 
        'banco av villas': r"(?=.*av)(?=.*villas)", 
        'banco union': r"(?=.*banco)(?=.*union)", 
        'banco bancoomeva': r"banco(o)?meva", 
        'banco falabella': r"(?=.*banco)(?=.*falabella)", 
        'banco pichincha': r"pichincha", 
        'banco w': r"banco\s?w", 
        'banco finandina': r"finandina", 
        'banco serfinanza': r"serfinanza",
        'banco bancamia': r"bancamia", 
        'banco credifinanciera': r"credifinanciera", 
        'banco cooperativo coopcentral':  r"(?=.*banco)(?=.*co(o)?perativo)|(?=.*co(o)?pcentral)",
        'banco bancoldex': r"bancoldex",
        'banco nequi':r"nequi",
        'banco de la republica': r"(?=.*banco)(?=.*republica)",
        #'banco qnt':r"(?=.*banco)(?=.*qnt)",
        
        
        #Secretarias_departamentales
        "secretaria educacion":  r"(?=.*secretaria)(?=.*educacion)",
        "secretaria ambiente": r"(?=.*secretaria)(?=.*ambiente)",
        "secretaria de cultura": r"(?=.*secretaria)(?=.*cultura)",
        "secretaria hacienda": r"(?=.*secretaria)(?=.*hacienda)",
        "secretaria seguridad": r"(?=.*secretaria)(?=.*seguridad)",
        "secretaria ambiente": r"(?=.*secretaria)(?=.*ambiente)",
        "secretaria desarrollo economico": r"(?=.*secretaria)(?=.*economico)",
        "secretaria desarrollo social": r"(?=.*secretaria)(?=.*desarrollo)(?=.*social)",
        "secretaria gobierno": r"(?=.*secretaria)(?=.*gobierno)",
        "secretaria integracion social": r"(?=.*secretaria)(?=.*integracion)",
        "secretaria movilidad": r"(?=.*secretaria)(?=.*movilidad)",
        "secretaria mujer": r"(?=.*secretaria)(?=.*mujer)",
        "secretaria planeacion": r"(?=.*secretaria)(?=.*planeacion)",
        "secretaria salud": r"(?=.*secretaria)(?=.*salud)",
        "secretaria seguridad": r"(?=.*secretaria)(?=.*seguridad)",
        "secretaria habitat": r"(?=.*secretaria)(?=.*habitat)",
        "secretaria general": r"(?=.*secretaria)(?=.*general)",
        "secretaria juridica distrital": r"(?=.*secretaria)(?=.*juridica)",
        "secretaria transito": r"(?=.*secretaria)(?=.*transito)", 
            }

### Fuzzy Matcher

In [7]:
import spacy
from spaczz.matcher import FuzzyMatcher
import pandas as pd
import re

nlp = spacy.blank("es")

In [8]:
doc_matches = []
matcher = FuzzyMatcher(nlp.vocab)
matcher.add("uariv", [nlp("uariv"), nlp("unidad atencion reparacion integral victimas"), nlp("unidad victimas")])
#pensiones 
matcher.add("colpensiones", [nlp("colpensiones")])
matcher.add("skandia afp", [nlp("skandia")])
matcher.add("proteccion afp", [nlp("proteccion afp")])
matcher.add("porvenir afp", [nlp("porvenir afp")])
#eps
matcher.add("asmet salud eps", [nlp("asmet salud"), nlp("asmet")])
matcher.add("asociacion indigena cauca aic eps", [nlp("aic epsi"), nlp("asociacion indigena cauca"), nlp("asociacion indigena cauca eps")])
matcher.add("aliansalud eps", [nlp("aliansalud")])
matcher.add("ambuq eps", [nlp("asociacion mutual barrios unidos de quibdo"), nlp("ambuq")])
matcher.add("anas wayuu eps", [nlp("anas wayuu")])
matcher.add("cajacopi eps", [nlp("cajacopi eps")])
matcher.add("capital salud eps", [nlp("capital salud")])
matcher.add("capresoca eps", [nlp("capresoca")])
matcher.add("coomeva eps", [nlp("coomeva eps")])
matcher.add("coosalud eps", [nlp("coosalud")])
matcher.add("comparta eps", [nlp("comparta eps")])
matcher.add("compensar eps", [nlp("compensar eps")])
matcher.add("comfachoco eps", [nlp("comfachoco eps")])
matcher.add("comfaca eps", [nlp("comfaca eps")])
matcher.add("comfacor eps", [nlp("comfacor eps")])
matcher.add("comfasucre eps", [nlp("comfasucre eps")])
matcher.add("comfanorte eps", [nlp("comfanorte eps")])
matcher.add("comfaoriente eps", [nlp("comfaoriente eps")])
matcher.add("comfamiliar eps", [nlp("comfamiliar eps")])
matcher.add("comfenalco eps", [nlp("comfenalco eps")])
matcher.add("comfandi eps", [nlp("comfandi eps")])
matcher.add("convida eps", [nlp("convida eps")])
matcher.add("cosmitet eps", [nlp("cosmitet eps")])
matcher.add("dusakawi eps", [nlp("dusakawi")])
matcher.add("ecoopsos eps", [nlp("ecoopsos")])
matcher.add("emssanar eps", [nlp("emssanar")])
matcher.add("famisanar eps", [nlp("famisanar")])
matcher.add("familiar colombia eps", [nlp("familiar colombia eps")])
matcher.add("mallamas eps", [nlp("mallamas eps")])
matcher.add("medimas eps", [nlp("medimas eps")])
matcher.add("mutualser eps", [nlp("mutualser eps")])
matcher.add("nueva eps", [nlp("nueva eps")])
matcher.add("pijaos salud eps", [nlp("pijaos salud")])
matcher.add("saludcoop", [nlp("saludcoop")])
matcher.add("salud bolivar eps", [nlp("salud bolivar eps")])
matcher.add("salud mia eps", [nlp("salud mia eps")])
matcher.add("salud total eps", [nlp("salud total")])
matcher.add("sanitas eps", [nlp("sanitas eps"), nlp("colsanitas eps")])
matcher.add("savia salud eps", [nlp("savia salud")])
matcher.add("servicio occidental de salud eps", [nlp("sos eps"), nlp("servicio occidental salud")])
matcher.add("sura eps", [nlp("sura eps")])

#ministerios
matcher.add("ministerio del interior", [nlp("ministerio interior"), nlp("mininterior")])
matcher.add("ministerio de relaciones exteriores", [nlp("ministerio relaciones exteriores"), nlp("cancilleria")])
matcher.add("ministerio de hacienda", [nlp("ministerio de hacienda"), nlp("minhacienda")])
matcher.add("ministerio de justicia y del derecho", [nlp("ministerio justicia derecho"),  nlp("minjusticia")])
matcher.add("ministerio de defensa", [nlp("ministerio defensa"),  nlp("mindefensa")])
matcher.add("ministerio de agricultura", [nlp("ministerio de agricultura"),  nlp("minagricultura")])
matcher.add("ministerio de salud", [nlp("ministerio de salud"),  nlp("minsalud")])
matcher.add("ministerio de trabajo", [nlp("ministerio de trabajo"),  nlp("mintrabajo")])
matcher.add("ministerio de minas y energia", [nlp("ministerio minas energia"),  nlp("minminas")])
matcher.add("ministerio de industria y comercio", [nlp("ministerio comercio industria"),  nlp("mincomercio")])
matcher.add("ministerio de educacion", [nlp("ministerio educacion"),  nlp("mineducacion")])
matcher.add("ministerio de ambiente", [nlp("ministerio ambiente"),  nlp("minambiente")])
matcher.add("ministerio de vivienda", [nlp("ministerio vivienda"),  nlp("minvivienda")])
matcher.add("ministerio de las tic", [nlp("ministerio tic"),  nlp("mintic")])
matcher.add("ministerio de transporte", [nlp("ministerio transporte"),  nlp("mintransporte")])
matcher.add("ministerio de cultura", [nlp("ministerio cultura"),  nlp("mincultura")])
matcher.add("ministerio de ciencia tecnologia e innovacion", [nlp("ministerio ciencia tecnologia innovacion"),  nlp("minciencias")])
matcher.add("ministerio de deporte", [nlp("ministerio de deporte"),  nlp("mindeporte")])
matcher.add("ministerio de igualdad", [nlp("ministerio de igualdad"),  nlp("minequidad")])


In [9]:
clasificaciones = {
    ("uariv", 83): "uariv",
    #pensiones
    ("colpensiones", 86): "colpensiones",
    ("skandia afp", 92): "skandia afp",
    ("proteccion afp", 96): "proteccion afp",
    ("porvenir afp", 87): "porvenir afp",
    #eps
    ("asmet salud eps", 86): "asmet salud eps",
    ("asociacion indigena cauca aic eps", 93): "asociacion indigena cauca aic eps",
    ("aliansalud eps", 91): "aliansalud eps",
    ("ambuq eps", 86): "ambuq eps",
    ("anas wayuu eps", 75): "anas wayuu eps",
    ("cajacopi eps", 75): "cajacopi eps",
    ("capital salud eps", 92): "capital salud eps",
    ("capresoca eps", 78): "capresoca eps",
    ("coomeva eps", 86): "coomeva eps",
    ("coosalud eps", 84): "coosalud eps",
    ("comparta eps", 80): "comparta eps",
    ("compensar eps", 92): "compensar eps",
    ("comfachoco eps", 83): "comfachoco eps",
    ("comfaca eps", 83): "comfaca eps",
    ("comfacor eps", 81): "comfacor eps",
    ("comfasucre eps", 84): "comfasucre eps",
    ("comfanorte eps", 84): "comfanorte eps",
    ("comfaoriente eps", 94): "comfaoriente eps",
    ("comfamiliar eps", 87): "comfamiliar eps",
    ("comfenalco eps", 93): "comfenalco eps",
    ("comfandi eps", 92): "comfandi eps",
    ("convida eps", 91): "convida eps",
    ("cosmitet eps", 80): "cosmitet eps",
    ("dusakawi eps", 75): "dusakawi eps",
    ("ecoopsos eps", 77): "ecoopsos eps",
    ("emssanar eps", 84): "emssanar eps",
    ("famisanar eps", 86): "famisanar eps",
    ("familiar colombia eps", 86): "familiar colombia eps",
    ("mallamas eps", 81): "mallamas eps",
    ("medimas eps", 91): "medimas eps",
    ("mutualser eps", 78): "mutualser eps",
    ("nueva eps", 89): "nueva eps",
    ("pijaos salud eps", 85): "pijaos salud eps",
    ("saludcoop", 76): "saludcoop",
    ("salud bolivar eps", 90): "salud bolivar eps",
    ("salud mia eps", 94): "salud mia eps",
    ("salud total eps", 87): "salud total eps",
    ("sanitas eps", 91): "sanitas eps",
    ("savia salud eps", 89): "savia salud eps",
    ("servicio occidental de salud eps", 89): "servicio occidental de salud eps",
    ("sura eps", 86): "sura eps",
    
    #ministerios
    ("ministerio del interior", 95): "ministerio del interior",
    ("ministerio de relaciones exteriores", 95): "ministerio de relaciones exteriores",
    ("ministerio de hacienda", 93): "ministerio de hacienda",
    ("ministerio de justicia y del derecho", 85): "ministerio de justicia y del derecho",
    ("ministerio de defensa", 84): "ministerio de defensa",
    ("ministerio de agricultura", 89): "ministerio de agricultura",
    ("ministerio de salud", 89): "ministerio de salud",
    ("ministerio de trabajo", 92): "ministerio de trabajo",
    ("ministerio de minas y energia", 97): "ministerio de minas y energia",
    ("ministerio de industria y comercio", 96): "ministerio de industria y comercio",
    ("ministerio de educacion", 87): "ministerio de educacion",
    ("ministerio de ambiente", 86): "ministerio de ambiente",
    ("ministerio de vivienda", 95): "ministerio de vivienda",
    ("ministerio de las tic", 99): "ministerio de las tic",
    ("ministerio de transporte", 95): "ministerio de transporte",
    ("ministerio de cultura", 99): "ministerio de cultura",
    ("ministerio de ciencia tecnologia e innovacion", 98): "ministerio de ciencia tecnologia e innovacion",
    ("ministerio de deporte", 92): "ministerio de deporte",
    ("ministerio de igualdad", 99): "ministerio de igualdad"
    
}

In [10]:
def clasificacion_entidades(texto_limpio, departamentos):
    entidades_encontradas = []
    for entidad, patron in entidades.items():
        if re.findall(patron, texto_limpio):
            entidades_encontradas.append(entidad)
            if entidad == "gobernacion":
                for departamento in departamentos:
                    if re.search(r'{}'.format(departamento), texto_limpio):
                        return entidad + " " + departamento
    return ', '.join(entidades_encontradas) if entidades_encontradas else "otro"

mach_values = []
ratio_values = []

for text in accionados["texto_limpio"]:
    doc = nlp(text)
    matches = matcher(doc)
    best_match_id = ""
    best_ratio = 0

    for match_id, start, end, ratio, pattern in matches:
        if ratio > best_ratio:
            best_match_id = match_id
            best_ratio = ratio

    mach_values.append(best_match_id)
    ratio_values.append(best_ratio)

# valores de "mach" y ratio
accionados["fuzzy"] = mach_values

# Clasificación basada en los ratios

for i in range(len(ratio_values)):
    ratio = ratio_values[i]
    classifications = []

    for key, value in clasificaciones.items():
        if mach_values[i] == key[0] and ratio > key[1]:
            classifications.append(value)

    if len(classifications) > 0:
        accionados.loc[i, "fuzzy"] = ", ".join(classifications)
    else:
        accionados.loc[i, "fuzzy"] = clasificacion_entidades(accionados.loc[i, "texto_limpio"], departamentos)


In [11]:
accionados["ratio"] = ratio_values

In [12]:
accionados.to_excel('fuzzy.xlsx', index=False)